# 환경설정 (for Colab)

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 작업 디렉토리 경로 변경
import os
notebook_path = '/content/drive/MyDrive/NewJJeans/HAND'
os.chdir(notebook_path)

Mounted at /content/drive


# Mecab 다운로드

In [ ]:
%cd mecab/Mecab-ko-for-Google-Colab

/content/drive/MyDrive/NewJJeans/HAND/mecab/Mecab-ko-for-Google-Colab


In [ ]:
!bash install_mecab-ko_on_colab_light_220429.sh # 1~3분정도 소요

In [ ]:
from konlpy.tag import Mecab
# 안되면 위에 **디렉토리변경** 한 다음에 하세요~!!!!!!!!!!!!!!
mecab = Mecab(dicpath='mecab-ko-dic-2.1.1-20180720')
word_list = ['아이즈원','권은비', '피프티 피프티', '침착맨', '투모로우바이투게더', '인스타그램', '미미로즈']
news = "‘김태리’ 라붐 소연, ‘놀면뭐하니?’ WSG워너비 최종 발탁. 라붐(LABOUM) 소연이 WSG워너비 멤버로 합류했다.지난 28일 방송된 MBC ‘놀면 뭐하니?’에서 WSG워너비 최종 멤버가 공개된 가운데, 소연은 최종 12인에 이름을 올리며 WSG워너비로 활동하게 됐다.앞서 WSG워너비 블라인드 오디션에서 김태리로 출연한 소연은 정승환의 ‘너였다면’을 열창해 3사 대표들에게 모두 합격점을 받는가 하면, 엘레나 킴은 “우리 팀의 리더가 돼 달라”라며 소연을 향한 러브콜을 보냈다.또한 전지현, 제시카 알바와 함께 ‘신선봉’ 조로 뭉쳤던 소연은 에코의 ‘행복한 나를’ 열창, 라붐의 메인 보컬답게 청아한 보이스와 완벽한 가창력은 물론, 조원들과 환상의 하모니를 선보여 다음 라운드로 진출했다.독보적인 보이스와 가창력을 뽐낸 김태리의 정체를 향한 여러 가지 추측과 관심이 쏟아진 가운데, 28일 방송된 ‘놀면 뭐하니?’에서 김태리의 정체가 소연으로 밝혀지면서 모두를 놀라게 했다.소연은 WSG워너비로 합류하게 된 소감으로 “예전에 MSG워너비가 부른 ‘상상더하기’를 보고 너무 놀라서 멤버들끼리 얼싸안았다. 그 프로젝트에 제가 들어왔다는 게 믿기지 않는다”라고 밝혔다.소연이 앞으로 WSG워너비 활동을 통해 어떤 행보를 걸을지 관심이 집중된다.한편 소연은 최근 국방TV ‘리얼 병영 톡! 행군기’의 고정 MC로 발탁되면서 앞으로도 다채로운 활동으로 대중을 만날 예정이다."
tokenizer = mecab.morphs

print(tokenizer(news))

['‘', '김', '태리', '’', '라', '붐', '소연', ',', '‘', '놀', '면', '뭐', '하', '니', '?', '’', 'WSG', '워너비', '최종', '발탁', '.', '라', '붐', '(', 'LABOUM', ')', '소연', '이', 'WSG', '워너비', '멤버', '로', '합류', '했', '다', '.', '지난', '28', '일', '방송', '된', 'MBC', '‘', '놀', '면', '뭐', '하', '니', '?', '’', '에서', 'WSG', '워너비', '최종', '멤버', '가', '공개', '된', '가운데', ',', '소연', '은', '최종', '12', '인', '에', '이름', '을', '올리', '며', 'WSG', '워너비', '로', '활동', '하', '게', '됐', '다', '.', '앞서', 'WSG', '워너비', '블라인드', '오디션', '에서', '김', '태리', '로', '출연', '한', '소연', '은', '정승환', '의', '‘', '너', '였', '다면', '’', '을', '열창', '해', '3', '사', '대표', '들', '에게', '모두', '합격', '점', '을', '받', '는가', '하', '면', ',', '엘레나', '킴', '은', '“', '우리', '팀', '의', '리더', '가', '돼', '달', '라', '”', '라며', '소연', '을', '향한', '러브', '콜', '을', '보냈', '다', '.', '또한', '전지현', ',', '제시카', '알바', '와', '함께', '‘', '신선봉', '’', '조', '로', '뭉쳤', '던', '소연', '은', '에코', '의', '‘', '행복', '한', '나', '를', '’', '열창', ',', '라', '붐', '의', '메인', '보컬', '답', '게', '청아', '한', '보이스', '와', '완벽', '한', '가창력', '은', 

# 0. Utils

In [ ]:
import sys
import csv
csv.field_size_limit(sys.maxsize)
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn import metrics
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.utils.data.dataset import Dataset
import numpy as np
import nltk # default word/sentence tokenizer
nltk.download('punkt')
from torch.utils.data import DataLoader
import shutil
# from konlpy.tag import Mecab

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
mecab = Mecab(dicpath='mecab-ko-dic-2.1.1-20180720').morphs
tokenizer("권은비")

['권은비']

In [ ]:
def get_evaluation(y_true, y_prob, list_metrics):
    y_pred = np.argmax(y_prob, -1)
    output = {}
    if 'accuracy' in list_metrics:
        output['accuracy'] = metrics.accuracy_score(y_true, y_pred)
    if 'loss' in list_metrics:
        try:
            output['loss'] = metrics.log_loss(y_true, y_prob)
        except ValueError:
            output['loss'] = -1
    if 'confusion_matrix' in list_metrics:
        output['confusion_matrix'] = str(metrics.confusion_matrix(y_true, y_pred))
    return output

def matrix_mul(input, weight, bias=False):
    feature_list = []
    for feature in input:
        feature = torch.mm(feature, weight)
        if isinstance(bias, torch.nn.parameter.Parameter):
            feature = feature + bias.expand(feature.size()[0], bias.size()[1])
        feature = torch.tanh(feature).unsqueeze(0)
        feature_list.append(feature)

    return torch.cat(feature_list, 0).squeeze()

def element_wise_mul(input1, input2):

    feature_list = []
    for feature_1, feature_2 in zip(input1, input2):
        feature_2 = feature_2.unsqueeze(1).expand_as(feature_1)
        feature = feature_1 * feature_2
        feature_list.append(feature.unsqueeze(0))
    output = torch.cat(feature_list, 0)

    return torch.sum(output, 0).unsqueeze(0)

def get_max_lengths(data_path):
    word_length_list = []
    sent_length_list = []
    with open(data_path) as csv_file:
        reader = csv.reader(csv_file, quotechar='"')
        for idx, line in enumerate(reader):
            text = ""
            for tx in line[1:]:
                text += tx.lower()
                text += " "
            sent_list = sent_tokenize(text)
            sent_length_list.append(len(sent_list))

            for sent in sent_list:
                word_list = word_tokenize(sent)
                word_length_list.append(len(word_list))

        sorted_word_length = sorted(word_length_list)
        sorted_sent_length = sorted(sent_length_list)

    return sorted_word_length[int(0.9*len(sorted_word_length))], sorted_sent_length[int(0.9*len(sorted_sent_length))]

# 1. Dataset

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data_path, dict_path, max_length_sentences=16, max_length_word=64, tokenizer=word_tokenize):
        super(MyDataset, self).__init__()

        texts, labels = [], []
        with open(data_path, encoding='utf-8-sig') as csv_file:
            reader = csv.reader(csv_file, quotechar='"')
            for idx, line in enumerate(reader):
                text = ""
                for tx in line[1:]:
                    text += tx.lower()
                    text += " "
                label = int(line[0])
                texts.append(text)
                labels.append(label)

        self.texts = texts
        self.labels = labels
        self.dict = pd.read_csv(filepath_or_buffer=dict_path, header=None, sep=" ", quoting=csv.QUOTE_NONE,
                                usecols=[0]).values
        self.dict = [word[0] for word in self.dict]
        self.max_length_sentences = max_length_sentences
        self.max_length_word = max_length_word
        self.num_classes = len(set(self.labels)) # classification class num

        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        label = self.labels[index]
        text = self.texts[index]
        document_encode = [
            [self.dict.index(word) if word in self.dict else -1 for word in self.tokenizer(sentences)] for sentences
            in
            sent_tokenize(text=text)] # get dictionary's index

        for sentences in document_encode: # padding (to max length word)
            if len(sentences) < self.max_length_word:
                extended_words = [-1 for _ in range(self.max_length_word - len(sentences))]
                sentences.extend(extended_words)

        if len(document_encode) < self.max_length_sentences:
            extended_sentences = [[-1 for _ in range(self.max_length_word)] for _ in
                                  range(self.max_length_sentences - len(document_encode))]
            document_encode.extend(extended_sentences)

        document_encode = [sentences[:self.max_length_word] for sentences in document_encode][
                          :self.max_length_sentences]

        document_encode = np.stack(arrays=document_encode, axis=0) # 여러개의 배열을 하나의 배열로 쌓아올림
        document_encode += 1 # -1을 0으로 맞추기

        return document_encode.astype(np.int64), label

# 2. Word Attention

In [ ]:
class WordAttNet(nn.Module):
    def __init__(self, dictionary_path, hidden_size=50):
        super(WordAttNet, self).__init__()
        dict = pd.read_csv(filepath_or_buffer=dictionary_path, header=None, sep=" ",
                           quoting=csv.QUOTE_NONE).values[:,1:] # 두번째 열부터 선택
        dict_len, embed_size = dict.shape
        dict_len += 1

        unknown_word = np.zeros((1, embed_size))
        # unknown_word 배열과 dict 배열을 합친 후, pytorch tensor로 변환함
        dict = torch.from_numpy(np.concatenate([unknown_word, dict], axis=0).astype(float))

        self.word_weight = nn.Parameter(torch.Tensor(2 * hidden_size, 2 * hidden_size))
        self.word_bias = nn.Parameter(torch.Tensor(1, 2 * hidden_size))
        self.context_weight = nn.Parameter(torch.Tensor(2 * hidden_size, 1))

        self.lookup = nn.Embedding(num_embeddings=dict_len, embedding_dim=embed_size).from_pretrained(dict)


        self.gru = nn.GRU(embed_size, hidden_size, bidirectional=True)
        self._create_weights(mean=0.0, std=0.05)

    def _create_weights(self, mean=0.0, std=0.05):

        self.word_weight.data.normal_(mean, std)
        self.context_weight.data.normal_(mean, std)

    def forward(self, input, hidden_state):

        output = self.lookup(input)
        f_output, h_output = self.gru(output.float(), hidden_state)  # feature output and hidden state output
        output = matrix_mul(f_output, self.word_weight, self.word_bias)
        output = matrix_mul(output, self.context_weight).permute(1,0)
        output = F.softmax(output, dim=-1)
        output = element_wise_mul(f_output,output.permute(1,0))

        return output, h_output

# 3. Sentence Attention

In [ ]:
class SentAttNet(nn.Module):
    def __init__(self, sent_hidden_size=50, word_hidden_size=50, num_classes=8):
        super(SentAttNet, self).__init__()

        self.sent_weight = nn.Parameter(torch.Tensor(2 * sent_hidden_size, 2 * sent_hidden_size))
        self.sent_bias = nn.Parameter(torch.Tensor(1, 2 * sent_hidden_size))
        self.context_weight = nn.Parameter(torch.Tensor(2 * sent_hidden_size, 1))

        self.gru = nn.GRU(2 * word_hidden_size, sent_hidden_size, bidirectional=True)
        self.fc = nn.Linear(2 * sent_hidden_size, num_classes)

        self._create_weights(mean=0.0, std=0.05)

    def _create_weights(self, mean=0.0, std=0.05):
        self.sent_weight.data.normal_(mean, std)
        self.context_weight.data.normal_(mean, std)

    def forward(self, input, hidden_state):

        f_output, h_output = self.gru(input, hidden_state)
        output = matrix_mul(f_output, self.sent_weight, self.sent_bias)
        output = matrix_mul(output, self.context_weight).permute(1, 0)
        output = F.softmax(output, dim=-1)
        output = element_wise_mul(f_output, output.permute(1, 0)).squeeze(0)
        output = self.fc(output)

        return output, h_output

# 4. Hierarchical Attention

In [ ]:
class HierAttNet(nn.Module):
    def __init__(self, word_hidden_size, sent_hidden_size, batch_size, num_classes, pretrained_dictionary_path,
                 max_sent_length, max_word_length):
        super(HierAttNet, self).__init__()
        self.batch_size = batch_size
        self.word_hidden_size = word_hidden_size
        self.sent_hidden_size = sent_hidden_size
        self.max_sent_length = max_sent_length
        self.max_word_length = max_word_length
        self.word_att_net = WordAttNet(pretrained_dictionary_path, word_hidden_size)
        self.sent_att_net = SentAttNet(sent_hidden_size, word_hidden_size, num_classes)
        self._init_hidden_state()

    def _init_hidden_state(self, last_batch_size=None):
        if last_batch_size:
            batch_size = last_batch_size
        else:
            batch_size = self.batch_size
        self.word_hidden_state = torch.zeros(2, batch_size, self.word_hidden_size)
        self.sent_hidden_state = torch.zeros(2, batch_size, self.sent_hidden_size)
        if torch.cuda.is_available():
            self.word_hidden_state = self.word_hidden_state.cuda()
            self.sent_hidden_state = self.sent_hidden_state.cuda()

    def forward(self, input):

        output_list = []
        input = input.permute(1, 0, 2)
        for i in input:
            output, self.word_hidden_state = self.word_att_net(i.permute(1, 0), self.word_hidden_state)
            output_list.append(output)
        output = torch.cat(output_list, 0)
        output, self.sent_hidden_state = self.sent_att_net(output, self.sent_hidden_state)

        return output

# parameters

In [ ]:
batch_size = 32
num_epochs = 20
learning_rate = 1e-3
####################################################################################
word_hidden_size = 64
sent_hidden_size = 64
####################################################################################
# train_data = 'data/Training_dataset.txt' # training data
train_data = 'data/new_training_0823.txt'
test_data = 'data/new_validation_0823.txt'
####################################################################################
tokenizer = Mecab(dicpath='mecab-ko-dic-2.1.1-20180720').nouns
dictionary_path = 'data/ko_w2v_version2.txt'

In [ ]:
training_params = {"batch_size": batch_size, "shuffle": True, "drop_last": True}
test_params = {'batch_size': batch_size, 'shuffle':False, "drop_last":False}
max_word_length, max_sent_length = 32, 20 #get_max_lengths(train_data) # 32 20 정도 하면 될듯 ..??
training_set = MyDataset(train_data, dictionary_path, max_sent_length, max_word_length, tokenizer=tokenizer)
training_generator = DataLoader(training_set, **training_params)
test_set = MyDataset(test_data, dictionary_path, max_sent_length, max_word_length, tokenizer=tokenizer)
test_generator = DataLoader(test_set, **test_params)

In [ ]:
max_word_length, max_sent_length

(30, 21)

# Model

In [ ]:
model = HierAttNet(
    word_hidden_size, sent_hidden_size,
    batch_size, training_set.num_classes,
    dictionary_path, max_sent_length, max_word_length
    )

# move model to GPU
if torch.cuda.is_available():
    model.cuda()

In [ ]:
""" 모델 epoch 평가 """
def eval_epoch(num_epoch, epoch, model, criterion, optimizer, data_loader):
    model.eval() # 모델을 평가 모드로 설정

    loss_ls = []
    losses = []
    te_label_ls = []
    te_pred_ls = []
    best_loss = 1e5
    #tqdm.notebook.tqdm
    with tqdm_notebook(total=len(data_loader), desc=f"Valid") as pbar:
        for feature, label in data_loader:
            num_sample = len(label)

            if torch.cuda.is_available():
                feature = feature.cuda()
                label = label.cuda()

            with torch.no_grad():
                model._init_hidden_state(num_sample)
                te_predictions = model(feature)

            te_loss = criterion(te_predictions, label)
            loss_ls.append(te_loss * num_sample)
            te_label_ls.extend(label.clone().cpu())
            te_pred_ls.append(te_predictions.clone().cpu())
            pbar.update(1)

        te_loss = sum(loss_ls) / test_set.__len__()
        te_pred = torch.cat(te_pred_ls, 0)
        label = np.array(te_label_ls)
        test_metrics = get_evaluation(label, te_pred.numpy(), list_metrics=["accuracy", "confusion_matrix"])
        confusion_matrix_str = test_metrics.get('confusion_matrix', None)

        print('============================== Validation ==============================')
        print("Epoch: {}/{}, Lr: {}, Loss: {}, Accuracy: {}".format(
            epoch + 1,
            num_epochs,
            optimizer.param_groups[0]['lr'],
            te_loss, test_metrics["accuracy"]))
        print('============================== Validation ==============================')

    return test_metrics["accuracy"], test_metrics["confusion_matrix"], te_loss

In [ ]:
from tqdm import tqdm, tqdm_notebook

# Hyperparameter Tuning

In [ ]:
# 손실함수와 optimizer를 정의합니다.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

best_loss = 1e5
best_epoch = 0
num_iter_per_epoch = len(training_generator)

# Resume from trained checkpoint

In [ ]:
from torch.utils.tensorboard import SummaryWriter
logdir = 'data/logs'
writer = SummaryWriter(logdir)

In [ ]:
model_path = 'data/checkpoints'
model_name_template = '/model_epoch{}_acc{}.pth'

# 체크포인트 파일 경로
checkpoint_path = 'data/checkpointsAfter_epoch5.pth'  # 체크포인트 파일 경로

checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu')) # using CPU
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch'] + 1  # 재개할 에포크 설정

# start_epoch  = 0
for epoch in range(start_epoch, num_epochs):
    params = {'num_epoch': num_epochs, 'epoch': epoch, 'model': model, 'optimizer': optimizer, 'criterion': criterion}
    num_iter_per_epoch = len(training_generator)

    for iter, (feature, label) in enumerate(training_generator):
        model.train() # 모델을 학습 모드로 설정
        if torch.cuda.is_available():
            feature = feature.cuda()
            label = label.cuda()
        model._init_hidden_state()
        predictions = model(feature)

        # 손실 함수를 이용하여 loss를 계산
        loss = criterion(predictions, label)

        # optimizer의 gradient 초기화
        optimizer.zero_grad()
        loss.backward() # gradient 계산
        optimizer.step() # model's parameter update using optimizer

        training_metrics = get_evaluation(label.cpu().numpy(), predictions.cpu().detach().numpy(), list_metrics=["accuracy"])

        if (iter+1) % 100 == 0:
            print("Epoch: {}/{}, Iteration: {}/{}, Lr: {}, Loss: {}, Accuracy: {}".format(
                epoch + 1,
                num_epochs,
                iter + 1,
                num_iter_per_epoch,
                optimizer.param_groups[0]['lr'],
                loss, training_metrics["accuracy"]))


        # 100번째 반복마다 저장 및 validation
        if (iter+1) % 400 == 0:
            # 모델 및 옵티마이저 상태 저장
            val_accuracy, confusion_matrix, te_loss = eval_epoch(**params, data_loader=test_generator)
            model_file_name = model_name_template.format(epoch, val_accuracy)
            torch.save({'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'epoch': epoch}, model_path + model_file_name)
            # training 및 validation Loss 저장
            writer.add_scalar("Train/Loss", loss, epoch * num_iter_per_epoch + iter)
            writer.add_scalar('Train/Accuracy', training_metrics["accuracy"], epoch * num_iter_per_epoch + iter)
            writer.add_scalar("Validation/Loss", te_loss, epoch * num_iter_per_epoch + iter)
            writer.add_scalar('Validation/Accuracy', val_accuracy, epoch * num_iter_per_epoch + iter)

    model_file_name = f'/After_epoch{epoch}.pth'
    torch.save({'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'epoch': epoch}, model_path + model_file_name)

# 학습 종료 후 close 메소드 호출
writer.flush()
writer.close()

In [ ]:
ls

 checkpoints/                   test.ipynb
'Copy of train_0822.ipynb'      train_0819.ipynb
'Copy of train.ipynb'           train_0820.ipynb
 data/                          train_0822.ipynb
 mecab/                         trained_models/
 mecab-ko-dic-2.1.1-20180720/   Training_0823_WED_newdata.ipynb
 Mecab-ko-for-Google-Colab/     training_mecab.ipynb
 old_data/                      train-mecab.ipynb
 predictions/                   train-mecab_Woojoo.ipynb
 reduce_vector_dim.ipynb        train-new.ipynb
 src/                           Untitled0.ipynb
 tensorboard/


In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2023-08-22 19:36:45.404673: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Address already in use
Port 6006 is in use by another program. Either identify and stop that program, or start the server with a different port.

In [ ]:
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir {logdir} --port 6000